In [1]:
from collections import defaultdict

import requests
import ricecooker
from bs4 import BeautifulSoup

from tessa_chef import get_modtype, get_resource_info
from tessa_chef import split_list_by_label
from tessa_chef import get_text, create_subpage_node
from tessa_chef import  TESSA_LANG_URL_MAP, TESSA_HOME_URL

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)

In [2]:
# top_level_url = TESSA_LANG_URL_MAP['fr']
tessa_wrtrees = []
for lang, top_level_url in TESSA_LANG_URL_MAP.items():
    response = requests.get(top_level_url)
    top_level_page = BeautifulSoup(response.content, "html5lib")
    # page_id = get_page_id(b)
    web_resource_tree = split_list_by_label(lang, top_level_page)
    tessa_wrtrees.append(web_resource_tree)







Processing TESSA page  EN Number of unfiltered activity links: 22
Append description: The subject resources are divi..



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66754
Recognizd standard module structure. Taking whole module:
         Content (oucontent): Module 1: Personal development – how self-esteem impacts on learning
 skipping module section li Section 1: Ways to explore who pupils are Unit content
 skipping module section li Section 2: Planning physical growth and development sessions Unit content
 skipping module section li Section 3: Exploring pupils' ideas about healthy living Unit content
 skipping module section li Section 4: Activities to support emotional well-being Unit content
 skipping module section li Section 5: Ways to promote spiritual well-being Unit content
Recognizd standard module structure. Taking whole module:
         Content (oucontent): Module 2: Exploring social development
 skipping module section li Section 1 : Exploring social ne

non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66698
Number of activity links: 21
Append description: Key resources have been produc..
Append description: Read or download individual re..



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66699
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66699
Number of activity links: 11
Append description: A handbook designed to support..
Append description: A short handbook for teachers ..
Append description: This handbook is designed for ..
Encountered subpage within subpage #### RECUSING VIA create_subpage_node ##########



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=80058
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=80058
Number of activity links: 21
Append descr




Processing TESSA page  FR Number of unfiltered activity links: 16
Ignoring activity of type resource 	startswith: Récapitulatif détaillé TESSA



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66857
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=66857
Number of activity links: 25
Append description: Lisez ou téléchargez des secti..
Append description: Lisez ou téléchargez des secti..
Append description: Lisez ou téléchargez des secti..



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=80922
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=80922
Number of activity links: 25
Append description: Lisez ou téléchargez des secti..
Append description: Lisez ou téléchargez des secti..
Append description: Lisez ou téléchargez des secti..



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=6

non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=81253
Number of activity links: 22
Append description: قم بقراءة أو تنزيل كل قسم من أ..
Append description: قم بقراءة أو تنزيل كل قسم من أ..
Append description: قم بقراءة أو تنزيل كل قسم من أ..
no hidden_subspan ... but proceeeding anyway
Recognizd standard module structure. Taking whole module:
         Content (oucontent): الوحدة رقم ١ :التقصّي في العدد و النمط
 skipping module section li القسم ١ : التعلم من خلال الألعاب.
 skipping module section li القسم ٢: الانماط في الرسم البياني للرقم.
 skipping module section li القسم ٣: وسائل حل المسائل الرياضية.
 skipping module section li القسم ٤ : رؤية عملية الضرب.
 skipping module section li القسم ٥ : عمل عملي باستخدام أشرطة و أقراص الكسور.
no hidden_subspan ... but proceeeding anyway
Recognizd standard module structure. Taking whole module:
         Content (oucontent): الوحدة رقم ٢ :التقصي في الشكل و الفراغ
 

non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=81299
Number of activity links: 21
Ignoring activity of type url 	startswith: Teaching Practice Supervisors’ Toolkit A Handbook 
Append description: تفضل بقراءة أو تنزيل البرامج و..
Ignoring activity of type url 	startswith: Teaching Practice Supervisors' Toolkit URL 
Ignoring activity of type url 	startswith: Welcome to the Teaching Practice Supervisors' Tool
Ignoring activity of type url 	startswith: 1. The Teaching Practice Supervisor URL 
Ignoring activity of type url 	startswith: 2. About TESSA URL 
Ignoring activity of type url 	startswith: 3. Recognising a good lesson URL 
Ignoring activity of type url 	startswith: 4. Planning and using TESSA materials with student
Ignoring activity of type url 	startswith: 5. Helping student teachers to carry out action pl
Ignoring activity of type url 	startswith: 6. Getting the most out of the school visit URL 
I

         Resource (resource): تنزيل الموارد الصوتية
         Resource (resource): تنزيل النص
         Resource (resource): تنزيل الملخص
         Resource (resource): تنزيل الموارد الصوتية
         Resource (resource): تنزيل النص
         Resource (resource): تنزيل الملخص
         Resource (resource): تنزيل الموارد الصوتية
         Resource (resource): تنزيل النص
         Resource (resource): تنزيل الملخص
         Resource (resource): تنزيل الموارد الصوتية
         Resource (resource): تنزيل النص
         Resource (resource): تنزيل الملخص
Encountered subpage within subpage #### RECUSING VIA create_subpage_node ##########



http://www.open.edu/openlearncreate/mod/subpage/view.php?id=81315
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=81315
Number of activity links: 13
Append description: في هذه الدراما، كانت الطالبة أ..
Ignoring activity of type resource 	startswith: تنزيل الموارد الصوتية
Ignoring acti

non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=82843
Number of activity links: 25
Append description: ..
Append description: Soma au pakua vipengele binafs..
Append description: ..
Append description: Soma au pakua vipengele binafs..
Append description: ..
Append description: Soma au pakua vipengele binafs..
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=82843
Number of activity links: 25
Append description: ..
no hidden_subspan ... but proceeeding anyway
Append description: Soma au pakua vipengele binafs..
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
Append description: ..
no hidden_subspan ... but proceeeding anyway
Append description: Soma

non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=82847
Number of activity links: 26
Append description: Kuna vitengo 15 vya Sayansi ya..
Append description: ..
Append description: Soma au pakua vipengele binafs..
Append description: ..
Append description: Soma au pakua vipengele binafs..
Append description: ..
Append description: Soma au pakua vipengele binafs..
non standard module, using fallback strategies...



Processing subpage http://www.open.edu/openlearncreate/mod/subpage/view.php?id=82847
Number of activity links: 26
Append description: Kuna vitengo 15 vya Sayansi ya..
Append description: ..
no hidden_subspan ... but proceeeding anyway
Append description: Soma au pakua vipengele binafs..
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proceeeding anyway
no hidden_subspan ... but proce

In [5]:

for web_resource_tree in tessa_wrtrees:
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        print('           desciption:', category['description'][0:60]+'..') # len(category['description']))
        
        for resource in category['children']:
            # print(resource)
            print('      - Resource (%s):' % resource['type'], resource['title'])
            for child in resource['children']:
                # print(child)
                print('         - Child (%s):' % child['type'], child['title'])
                
    print('\n\n')



----
WEB RESOURCE TREE: title: TESSA (EN)   len(children) = 4
      description: TESSA materials have been created and developed by experts w..


KeyError: 'url'

In [4]:

REJECT_TITLES = [
    # Overview pages that link to other modules --- TODO: scrape manually and process to remove link colors
    'Curriculum framework',
    'Résumé et récapitulatif des matériels TESSA',
    'طار المناهج',
    'Mfumo wa mtaala',
    # 'Download the complete Pan-Africa English library' # keeping bcs we can handle pdf
]


def filter_unwanted_categories(tree):
    filtered_resource_tree = tree.copy()
    filtered_resource_tree['children'] = []
    
    for category in tree['children']:
        
        filter_exclue_match_found = False
        for reject_title in REJECT_TITLES:
            #print('ZZZ'+reject_title+'ZZZ')
            if reject_title in category['title']:
                filter_exclue_match_found = True
                print('filter_exclue_match_found', reject_title)

        if filter_exclue_match_found:
            pass
        else:
            filtered_resource_tree['children'].append(category)

    return filtered_resource_tree

filtered_tessa_wrtrees = []
for wrt in tessa_wrtrees:
    filtered_wrt = filter_unwanted_categories(wrt)
    filtered_tessa_wrtrees.append(filtered_wrt)


for web_resource_tree in filtered_tessa_wrtrees:
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        print('           desciption:', category['description'][0:60]+'..') # len(category['description']))
        
        for resource in category['children']:
            # print(resource)
            print('      - Resource (%s):' % resource['type'], resource['title'])
            for child in resource['children']:
                # print(child)
                print('         - Child (%s):' % child['type'], child['title'])
                
    print('\n\n')

            

filter_exclue_match_found Curriculum framework
filter_exclue_match_found Résumé et récapitulatif des matériels TESSA
filter_exclue_match_found طار المناهج
filter_exclue_match_found Mfumo wa mtaala
----
WEB RESOURCE TREE: title: TESSA (EN)   len(children) = 3
      description: TESSA materials have been created and developed by experts w..
   - Category   title: Subject resources
           desciption:  The subject resources are divided into six subject areas wh..
      - Resource (TessaSubpage): Life Skills (primary)
         - Child (oucontent): Module 1: Personal development – how self-esteem impacts on learning
         - Child (oucontent): Module 2: Exploring social development
         - Child (oucontent): Module 3: Community issues and citizenship
      - Resource (TessaSubpage): Literacy (primary)
         - Child (oucontent): Module 1: Reading and writing for a range of purposes
         - Child (oucontent): Module 2: Using community voices in your classroom
         - Child (o